<a href="https://colab.research.google.com/github/proteus21/DATA-SCIENCE-STUDY/blob/main/Computer%20Vision/Tensorflow_Exercises_part_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow - Data pipelines - Fifth Exercise (20 minutes)
@author Tomasz Skrzypczyk

Solved by Boguslaw Konefal

In [ ]:
# do not modify this cell
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data = load_iris()
columns = data.feature_names

X = data.data
y = data.target


X_train, X_test, y_train, y_test = train_test_split(X, y,test_size= 0.1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,test_size=0.1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# First attempt to tf.data.Dataset

Lets create a dataset - it works sth like pandas Dataframe, but its more functional.


Create your dataset by using the tf.data.Dataset.from_tensor_slices function.
Pass a tuple - your X and y.

Create both training, validation and test datasets.




Next step: shuffle the data using the shuffle method. Warning: it returns the shuffled dataset, so you must assign it back to the original variable. See some examples here:  https://stackoverflow.com/questions/50437234/tensorflow-dataset-shuffle-then-batch-or-batch-then-shuffle
Also, remember which datasets you actually want to shuffle, do we need it for validation or test set?

Last step: Split your data in batches, its kind of mandatory. Use the batch method, set batch_size to 32.





In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(9)
train_ds = train_ds.batch(32)


val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_ds = val_ds.batch(15)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_ds = test_ds.batch(46)

In [ ]:
# do not modify this cell

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(3, input_shape=(4,)))

model.compile(loss="sparse_categorical_crossentropy")

model.evaluate(train_ds)
model.evaluate(val_ds)
model.evaluate(test_ds)



1/1 [==============================] - 0s 6ms/step - loss: 8.0056


8.005592346191406

# Second approach - using a generator


This method is slightly more difficult, but more efficient and gives you more options.


Write a generator function that yields a tuple of data - single x and y

```
def my_data_generator(X, y):


    # some code, very often in a for loop
    for ...
        # some code here

        yield sample, label

```

Create your Dataset by using your data generator (https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_generator)

```
ds = = tf.data.Dataset.from_generator(
     my_data_generator,
     output_signature=(
         tf.TensorSpec(shape=(4,), dtype=tf.int32),
         tf.TensorSpec(shape=(1,), dtype=tf.int32)))


```
You will be asked to define the output_signature parameter,
try this approach 

`output_signature=(
         tf.TensorSpec(shape=(shape of a single sample), dtype=datatype),
         tf.TensorSpec(shape=(shape of a single label), dtype=datatype)))`

Our samples are expressed with floats, remember iris data?, and the labels - its a single integer.









In [ ]:
# Create your generator function and then create your 3 datasets again using the generator, 
# one generator for 3 datasets, remember the shuffling and batching operations!!



def my_data_generator(X,Y):

  def my_data_generator_tr():
      for x,y in zip(X, Y):
        yield x, (y,)

  return my_data_generator_tr



train_ds = tf.data.Dataset.from_generator(
     my_data_generator(X_train, y_train),
     output_signature=(
         tf.TensorSpec(shape=(4,), dtype=tf.float32),
         tf.TensorSpec(shape=(1,), dtype=tf.int32)))
train_ds = train_ds.shuffle(9)
train_ds = train_ds.batch(32)

val_ds = tf.data.Dataset.from_generator(
     my_data_generator(X_val, y_val),
     output_signature=(
         tf.TensorSpec(shape=(4,), dtype=tf.float32),
         tf.TensorSpec(shape=(1,), dtype=tf.int32)))
val_ds = val_ds.batch(15)

test_ds = tf.data.Dataset.from_generator(
     my_data_generator(X_test, y_test),
     output_signature=(
         tf.TensorSpec(shape=(4,), dtype=tf.float32),
         tf.TensorSpec(shape=(1,), dtype=tf.int32)))
test_ds = test_ds.batch(46)


In [ ]:
# check if this cell compiles to see if you have defined your datasets correctly


model.evaluate(train_ds)
model.evaluate(val_ds)
model.evaluate(test_ds)

1/1 [==============================] - 0s 75ms/step - loss: 8.0056


8.005592346191406